In [13]:
import tensorflow as tf

In [14]:
# additional imports

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model

In [15]:
# Load in the data
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()
print("x_train.shape:", x_train.shape)
print("y_train.shape", y_train.shape)

x_train.shape: (50000, 32, 32, 3)
y_train.shape (50000,)


In [16]:
# number of classes
K = len(set(y_train))
print("number of classes:", K)

number of classes: 10


In [17]:
# Build the model using the functional API
def create_model():
  i = Input(shape=x_train[0].shape)

  x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
  x = BatchNormalization()(x)
  x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)

  x = Flatten()(x)
  x = Dropout(0.2)(x)
  x = Dense(1024, activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(K, activation='softmax')(x)

  model = Model(i, x)
  return model

In [18]:
#strategy = tf.distribute.MirroredStrategy()
strategy = tf.distribute.experimental.CentralStorageStrategy()

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:GPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:GPU:0'


In [19]:
print(f'Number of devices: {strategy.num_replicas_in_sync}')

Number of devices: 1


In [20]:
with strategy.scope():
  model = create_model()

  model.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [21]:
# Fit
r = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)

Epoch 1/5
1563/1563 [==============================] - 12s 7ms/step - accuracy: 0.5602 - loss: 1.2749 - val_accuracy: 0.5982 - val_loss: 1.1895
Epoch 2/5
1563/1563 [==============================] - 11s 7ms/step - accuracy: 0.7045 - loss: 0.8518 - val_accuracy: 0.6974 - val_loss: 0.8719
Epoch 3/5
1563/1563 [==============================] - 11s 7ms/step - accuracy: 0.7610 - loss: 0.6973 - val_accuracy: 0.7634 - val_loss: 0.7075
Epoch 4/5
1563/1563 [==============================] - 11s 7ms/step - accuracy: 0.7987 - loss: 0.5842 - val_accuracy: 0.7470 - val_loss: 0.7730
Epoch 5/5
1563/1563 [==============================] - 11s 7ms/step - accuracy: 0.8250 - loss: 0.5117 - val_accuracy: 0.8016 - val_loss: 0.6053


In [10]:
50000/391

127.8772378516624

In [11]:
10000/79

126.58227848101266

In [ ]:
# Compare this to non-distributed training
model2 = create_model()
model2.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
r = model2.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)

Epoch 1/5
1563/1563 [==============================] - 11s 7ms/step - loss: 1.3265 - accuracy: 0.5414 - val_loss: 1.1715 - val_accuracy: 0.6071
Epoch 2/5
 491/1563 [========>.....................] - ETA: 6s - loss: 0.9195 - accuracy: 0.6795